In [ ]:
# disregard this cell
#!nvidia-smi
#import os

#os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [ ]:
# add stuff here to make it work in colab...

In [ ]:
!python3 prepare_data.py shakespeare_input.txt shake

In [ ]:
import pickle

import tensorflow as tf
from prepare_data import parse_seq


# data
bs = 128
seq_len = 200
data = tf.data.TFRecordDataset("shake.tfrecords")
data = data.map(lambda x: parse_seq(x, seq_len))
data = data.shuffle(46000).batch(bs).repeat()

vocab = pickle.load(open("shake_vocab", mode="rb"))
vocab_size = len(vocab)
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab_size)

68


In [ ]:
# don't run this cell lol
# it prints all of the data forever
for ind, thing in enumerate(data):
    inds = thing[0].numpy()
    to_chars = "".join([ind_to_ch[ind] for ind in inds])
    #print(ind)
    print(to_chars)
    #print()
    break

<S>ell, what worst?

Messenger:
The nature of bad news infects the teller.

MARK ANTONY:
When it concerns the fool or coward. On:
Things that are past are done with me. 'Tis thus:
Who tells me true, tho


In [ ]:
# model
n_h = 512
w_xh = tf.Variable(tf.initializers.glorot_uniform()([vocab_size, n_h]))
w_hh = tf.Variable(tf.initializers.glorot_uniform()([n_h, n_h]))
b_h = tf.Variable(tf.zeros([n_h]))

w_ho = tf.Variable(tf.initializers.glorot_uniform()([n_h, vocab_size]))
b_o = tf.Variable(tf.zeros([vocab_size]))

all_vars = [w_xh, w_hh, b_h, w_ho, b_o]

In [ ]:
# training
# somewhat arbitrary number of steps
steps = 20*35000 // bs
opt = tf.optimizers.Adam()
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)


@tf.function
def run_rnn_on_seq(seq_batch):
    with tf.GradientTape() as tape:
        state = tf.zeros([tf.shape(seq_batch)[0], n_h])
        total_loss = tf.constant(0.)

        for time_step in tf.range(tf.shape(seq_batch)[1] - 1):
            inp_here = tf.one_hot(seq_batch[:, time_step], vocab_size)
            state = tf.nn.tanh(tf.matmul(inp_here, w_xh) + tf.matmul(state, w_hh) + b_h)
            logits = tf.matmul(state, w_ho) + b_o

            loss_here = loss_fn(seq_batch[:, time_step+1], logits)
            total_loss += loss_here
            
        total_loss /= tf.cast(tf.shape(seq_batch)[1] - 1, tf.float32)
    grads = tape.gradient(total_loss, all_vars)
    
    # this is gradient clipping
    glob_norm = tf.linalg.global_norm(grads)
    grads = [g/glob_norm for g in grads]
    
    opt.apply_gradients(zip(grads, all_vars))

    return total_loss


# alternative function that, instead of summing up the loss at each time step,
# builds a "loss sequence" over time
# in principle, we could just build a list with one element per time step
# but this will not work with tf.function (tensors and python lists don't play
# together very well) so we use a thing called TensorArray
@tf.function
def run_rnn_on_seq_alternative(seq_batch):
    with tf.GradientTape() as tape:
        state = tf.zeros([tf.shape(seq_batch)[0], n_h])
        # this is where the per-time step losses will go
        losses = tf.TensorArray(tf.float32, size=tf.shape(seq_batch)[1]-1)

        for time_step in tf.range(tf.shape(seq_batch)[1] - 1):
            inp_here = tf.one_hot(seq_batch[:, time_step], vocab_size)  # batch x vocab
            state = tf.nn.tanh(tf.matmul(inp_here, w_xh) + tf.matmul(state, w_hh) + b_h)
            logits = tf.matmul(state, w_ho) + b_o

            # batch-size loss tensor for this time step
            # could still use loss_fn here as in the function above, but that would average over the
            # batch already. I would like to keep the batch axis here to show how this could
            # be used with a mask (see below). that's why this uses tf.nn.sparse...
            loss_here = tf.nn.sparse_softmax_cross_entropy_with_logits(seq_batch[:, time_step+1], logits)
            
            losses = losses.write(time_step, loss_here)
        losses = losses.stack() # put them together in a tensor, but it will be time x batch
        losses = tf.transpose(losses, [1, 0]) # not really necessary, but transpose to batch x time
        
        # if, say, we had a batch x time mask tensor, we could multiply it with the loss here...
        #losses = losses * mask
        
        total_loss = tf.reduce_mean(losses) # average over batch and time axes
            
        
    grads = tape.gradient(total_loss, all_vars)
    glob_norm = tf.linalg.global_norm(grads)
    grads = [g/glob_norm for g in grads]
    opt.apply_gradients(zip(grads, all_vars))

    return total_loss

In [ ]:
for step, seqs in enumerate(data):
    xent_avg = run_rnn_on_seq(seqs)

    if not step % 200:
        print("Step: {} Loss: {}".format(step, xent_avg))
        print()
        

    if step > steps:
        break

In [ ]:
import numpy as np

def sample(n_steps):
    state = tf.zeros([1, n_h])
    gen = [0]

    for step in range(n_steps):
        state = tf.nn.tanh(tf.matmul(tf.one_hot(gen[-1:], depth=vocab_size), w_xh) + tf.matmul(state, w_hh) + b_h)
        probs = tf.nn.softmax(tf.matmul(state, w_ho) + b_o).numpy()[0]
        #gen.append(np.argmax(probs))  # use argmax instead of choice if you want
        gen.append(np.random.choice(vocab_size, p=probs))
    return "".join([ind_to_ch[ind] for ind in gen])
        
agg = sample(2000)

In [ ]:
print(agg)